## Install the Package and Data

In [1]:
#This notebook was created using this tutorial:
    #https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

In [2]:
#Import the Natural Language Toolkit and download sample tweets from the package
import nltk
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\abiel\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

## Tokenize the Data

In [3]:
# import the twitter_samples
from nltk.corpus import twitter_samples

#This will import three datasets from NLTK that contain various tweets to train and test the model:
    #negative_tweets.json: 5000 tweets with negative sentiments
    #positive_tweets.json: 5000 tweets with positive sentiments
    #tweets.20150430-223406.json: 20000 tweets with no sentiments

In [4]:
#Create variables for positive_tweets, negative_tweets, and text
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [5]:
#import the punkt resource to help tokenize words and sentences
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#tokenize the tweets in the twitter samples dataset
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [7]:
#print an example - be sure to add [0] to the end of the previous line of code
#print(tweet_tokens[0])

## Normalize the Data

In [8]:
#Normalization in NLP is the process of converting a word to its canonical form (convert all tenses of the word to the same tense. ex: ran, runs, running to run)
#Normalization helps group together words with the same meaning but different forms. 
#Without normalization, “ran”, “runs”, and “running” would be treated as different words, even though you may want them to be treated as the same word. 

In [9]:
#This Python notebook will use lemmatization which normalizes a word in the context of vocabulary and morphological analysis.
#Download two neccessary resources for this normalizaton process
nltk.download('wordnet')  #this is a lexical database for the English language which helps determine the base word
nltk.download('averaged_perceptron_tagger')  #this determines the context of the word in a sentence

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\abiel\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
#Download a tagging algorithm and print the tags for a set of tweets
from nltk.tag import pos_tag
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [11]:
#From the list of tags, here is the list of the most common items and their meaning:
#NNP: Noun, proper, singular
#NN: Noun, common, singular or mass
#IN: Preposition or conjunction, subordinating
#VBG: Verb, gerund or present participle
#VBN: Verb, past participle

In [12]:
#incorporate this into a function that normalizes a sentence:
    #first generate the tags for each token in the text
    #then lemmatize each word using the tag

#import the WordNetLemmatizer class and initialize it to a variable, lemmatizer
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

#this function gets the position tag of each token of a tweet
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []

#if the tag starts with NN, the token is assigned as a noun. if the tag starts with VB, the token is assigned as a verb.
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

#view the results and notice that the verb 'being' changes to its root form, 'be', and the noun 'members' changes to 'member'.
#print(lemmatize_sentence(tweet_tokens[0]))



['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


## Remove Noise from the Data

In [ ]:
#Noise is any part of the text that does not add meaning or information to data.
#for this analysis, noise includes: Hyperlinks, Twitter handles ('@') in replies, Punctuation and special characters


Hyperlinks

In [ ]:
#To remove hyperlinks, first search for a substring that matches a URL starting with http:// or https://
#and followed by letters, numbers, or special characters. 
#Once a pattern is matched, the .sub() method replaces it with an empty string.

In [14]:
#This code creates a remove_noise() function that removes noise and incorporates the normalization and lemmatization mentioned in the previous section. 

import re, string

#The code takes two arguments: the tweet tokens and the tuple of stop words.
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

#The code then uses a loop to remove the noise from the dataset. 
    for token, tag in pos_tag(tweet_tokens):
        
#To remove hyperlinks, the code first searches for a substring that matches a URL starting with http:// or https://, followed by letters, numbers, or special characters.
#Once a pattern is matched, the .sub() method replaces it with an empty string, or ''.
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)

#Similarly, to remove @ mentions, the code substitutes the relevant part of text using regular expressions. 
#The code uses the re library to search @ symbols, followed by numbers, letters, or _, and replaces them with an empty string.
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [15]:
#and, remove stop words using a built-in set of stop words in NLTK.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abiel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
#use the .words() method to get a list of stop words in English
#To test the function, run it on a sample tweet
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

#the function removes all @ mentions, stop words, and converts the words to lowercase

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [17]:
#use the remove_noise() function to clean the positive and negative tweets

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [20]:
#check some samples from the positive and negative tweets lists
print(remove_noise(positive_cleaned_tokens_list[6], stop_words))
print(remove_noise(negative_cleaned_tokens_list[0], stop_words))

['like', 'keep', 'lovely', 'customer', 'wait', 'long', 'hope', 'enjoy', 'happy', 'friday', 'lwwf', ':)']
['hopeless', 'tmr', ':(']


In [21]:
#compare the original tokens to the cleaned tokens for a sample positive tweet
print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


In [23]:
#compare the original tokens to the cleaned tokens for a sample positive tweet
print(negative_tweet_tokens[450])
print(negative_cleaned_tokens_list[450])

['@LittleMix', 'have', 'fun', 'in', 'Germany', ':(', 'I', "can't", 'be', 'there', '😭']
['fun', 'germany', ':(', "can't", '😭']


## Determine Word Density

In [24]:
#define a generator function, named get_all_words, that takes a list of tweets as an argument to provide a list of words in all of the tweet tokens joined

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

#This has compiled all words in the sample of tweets

In [25]:
#find out which are the most common words using the FreqDist class of NLTK

from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


## Preparing Data for the Sentiment Analysis Model

In [ ]:
#Sentiment analysis is a process of identifying an attitude of the author on a topic that is being written about. 

#For this model, create a training data set for supervised learning machine learning 
#this requires each dataset (positive tweets and negative tweets) to be associated with a “sentiment” for training. 
#for simplicity - this model will only use two sentiments: positive and negative
